In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

C:\Users\akrivia\miniconda3\envs\genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embedding = GoogleGenerativeAIEmbeddings(google_api_key=os.getenv("GOOGLE_API_KEY"),model="models/embedding-001")
llm = ChatGoogleGenerativeAI(google_api_key = os.getenv("GOOGLE_API_KEY"),model="gemini-2.5-flash")

In [4]:
llm.invoke("hi").content

'Hi there! How can I help you today?'

## Simple AI Assistant 

In [5]:
while True:
    question = input("Type your Question.\nIf you want to quit the chat write q")
    if question != "q":
        print(llm.invoke(question).content)
    else:
        print("Goodbye take care Yourself")
        break

Type your Question.
If you want to quit the chat write q hi


Hi there! How can I help you today?


Type your Question.
If you want to quit the chat write q q


Goodbye take care Yourself


### memory integration

In [6]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [7]:
store = {}

In [8]:
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [9]:
config = {"configurable":{'session_id':'firstchat'}}

In [10]:
model_with_memory = RunnableWithMessageHistory(llm,get_session_history)

In [11]:
model_with_memory.invoke(("Hi, I'm Lalit"),config = config).content

'Hi Lalit! Nice to meet you.\n\nHow can I assist you today?'

In [12]:
model_with_memory.invoke(("who am i ?"),config = config).content

'That\'s a profound and deeply personal question, Lalit!\n\nAs an AI, I don\'t have access to your personal history, memories, feelings, or experiences. I don\'t know who you are as an individual human being with all your unique qualities, relationships, and journey.\n\n"Who am I?" is a question that people often explore throughout their lives. It typically refers to:\n\n*   **Your unique identity:** Your personality, values, beliefs, passions, strengths, weaknesses.\n*   **Your history:** Your past experiences, the people who shaped you, the challenges you\'ve overcome.\n*   **Your relationships:** How you connect with others, your roles in their lives (friend, family member, colleague).\n*   **Your purpose:** What drives you, what you want to achieve, what gives your life meaning.\n*   **Your aspirations:** Who you want to become in the future.\n\nWhile I can\'t tell you who you are, I can help you think about it. If you\'d like, we could explore some prompts or ideas that might help

## RAG base AI Assistant

In [13]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader,PyM
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma

In [16]:
loader = DirectoryLoader("./data",glob="./*.pdf",loader_cls=PyPDFLoader)
pdf_loader = PyPDFLoader("data/intro-to-algo-4e.pdf").load()

In [17]:
# docs = loader.load()

docs = pdf_loader.load()

Previous trailer cannot be read: ("'NumberObject' object is not subscriptable",)
parsing for Object Streams


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 50,chunk_overlap = 10,length_function = len)

In [ ]:
new_docs = text_splitter.split_documents(documents = docs)
doc_strings = [doc.page_content for doc in new_docs]

In [ ]:
docs

In [ ]:
db = Chroma.from_documents(new_docs,embedding=embedding)
retriever = db.as_retriever(search_kwargs = {"k":4})

### Agents

In [18]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [19]:
wiki_api_wrapper = WikipediaAPIWrapper()

In [20]:
tool  = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [21]:
tool.name

'wikipedia'

In [23]:
tool.run({"query":"ai"})

C:\Users\akrivia\miniconda3\envs\genai\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\akrivia\miniconda3\envs\genai\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


'Page: Artificial intelligence\nSummary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.\nHigh-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into gener

In [24]:
from langchain_community.tools import YouTubeSearchTool

In [25]:
yt = YouTubeSearchTool()

In [26]:
yt.name

'youtube_search'

In [27]:
yt.args

{'query': {'title': 'Query', 'type': 'string'}}

In [30]:
yt.run({"query":"indian drama"})

"['https://www.youtube.com/shorts/c9ooPiwDdio', 'https://www.youtube.com/shorts/JhAVOTV1k_Q']"

In [31]:
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [33]:
agent = initialize_agent(tools=tool,llm=llm,agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose = True)

AttributeError: 'tuple' object has no attribute 'is_single_input'

In [34]:
agent.run("what is the current GDP of india.")

NameError: name 'agent' is not defined